Makes Table 1 & 2 and takes all of the preliminary SNeCO data and organizes it, preparing it for when we begin making intensity measurements in section 3.

To run this notebook we need '2.SNeCOMasterCatManualClean.txt' (see prepwork below). Once this prepwork is done, it is not needed to repeat unless we update the sample / redownload the OSC.

We also need "0.PHANGS_v1p6.csv" to pull other galaxy information from the survey. 
We also use "0.PHANGSMapFiles.txt" to pull the telescope orientations used for each galaxy.

      *PREPWORK* 
      
After looking at the galaxy images from #2. We find we need to remove one SN that isnt within the region covered by the PHANGS-ALMA footprint. We remove this (SN2003gd from NGC0628) and also the 6 candidate SNe, the one duplicate entry, and the LRN, and begin to perform some rudimentary stats in small tables.

Before we run this code we do the following prep:

We take the 2.SNeCOMasterCat.txt file and manually remove the following SNe:

Previously (& Permanently) Removed
AT2020nqq   (LRN) 
PTSS-19clju (Duplicate)

Recently commented out
AT2018eoq   (Candidate)
AT2019pck   (Candidate)
AT2019npi   (Candidate)
AT2019npd   (Candidate)
AT2020hol   (Candidate)
AT2020juh   (Candidate)
AT2020cwh   (Candidate)
SN2003gd    (Out of Map ngc0628)
SN2005af    (Out of Map ngc4945)
ngc5236,PS  (Not in Map & Also not good data on this one)

*also shortened type IIn, IIb to II and type Ia-91bg to Ia, etc.

Then we save this as 2.SNeCOMasterCatManualClean.txt

We turn this file into a dataframe and use to make basic tables of stats.
**********************
    
    *MORE PREPWORK* Changed 'name' column to 'galaxy' & 'eso097-013' to 'circinus' in 0.PHANGS_v1p6.csv file
    

In [1]:
# IMPORT PACKAGES

import numpy as np
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table, join
from astropy.wcs import WCS
from astropy.io import ascii
import astropy
from astropy.io import fits



In [2]:
# Prep table 1 - OSC Objects

OSC_File = '../Data/2.SNeCOMasterCat.txt'
OSC_Objects = Table.read(OSC_File, format='csv') 

Clean_File = '../Data/2.SNeCOMasterCatFinal.csv'
SNeSample = Table.read(Clean_File, format='csv')

ra, dec, papers, flags = [],[],[],[]

for i in range(len(OSC_Objects['#Galaxy'])):
    
    ra.append(round(OSC_Objects["Ra"][i],4))
    dec.append(round(OSC_Objects["Dec"][i],4))
    papers.append('?')
    
    if OSC_Objects['Supernova'][i] in SNeSample['SNName']:
        flags.append('\checkmark')
    
    else: flags.append('-')
                     
OSCtable = Table({'Supernova'            : OSC_Objects['Supernova'],
                       'Type'            : OSC_Objects['Type'],
                       'Galaxy'          : OSC_Objects['#Galaxy'],
                       'Ra'              : ra,
                       'Dec'             : dec,
                       'Reference Paper' : papers,            
                       'SNeCOSample'     : flags})

ascii.write(OSCtable, '../Data/3.OSCTable.tex', format='latex', fast_writer=False, overwrite=True)


In [3]:
OSCtable

Supernova,Type,Galaxy,Ra,Dec,Reference Paper,SNeCOSample
str11,str12,str8,float64,float64,str1,str10
AT2020juh,Candidate,circinus,213.3379,-65.3413,?,-
SN1996cr,II,circinus,213.2918,-65.3457,?,\checkmark
AT2019npi,Candidate,ngc0247,11.7453,-20.7083,?,-
AT2019npd,Candidate,ngc0253,11.7363,-25.3768,?,-
AT2020hol,Candidate,ngc0253,11.8539,-25.3571,?,-
AT2019pck,Candidate,ngc0253,11.8514,-25.2261,?,-
SN1940E,I,ngc0253,11.8783,-25.2934,?,\checkmark
SN2019qyl,IIn/LBV,ngc0300,13.7399,-37.6444,?,\checkmark
SN2013ej,II,ngc0628,24.2007,15.7586,?,\checkmark


In [4]:
# Turn file into dataStructure and clean

dataFile      = '../Data/2.SNeCOMasterCatManualClean.txt'
galaxy, SNname, SNtype = np.genfromtxt(dataFile,  dtype=str, unpack=True, delimiter = ',', usecols=(0,1,2))
ra, dec, xVal, yVal, co_21_int, co_21_err, co_21_ew, beamsize  = np.genfromtxt(dataFile,  dtype=float, unpack=True, delimiter = ',', usecols=(3,4,5,6,7,8,9,10))

# remove spaces and " " from SN data
galaxy = [x.strip() for x in galaxy]
#SNname = [x[2:-2] for x in SNname]
SNtype = [x.strip() for x in SNtype]

# build table 
data = Table({'galaxy'       : galaxy,
              'SN_name'      : SNname,
              'SN_type'      : SNtype,
              'SN_ra'        : ra,
              'SN_dec'       : dec,
              'SN_xCoord'    : xVal,
              'SN_yCoord'    : yVal,
              'SN_co21int'   : co_21_int,
              'SN_co21err'   : co_21_err,
              'SN_co21ew'    : co_21_ew,
              'map_beamsize' : beamsize,
             },
             names=['galaxy', 'SN_name','SN_type','SN_ra','SN_dec','SN_xCoord', 'SN_yCoord', 'SN_co21int', 'SN_co21err', "SN_co21ew", 'map_beamsize'])


In [5]:
# Pull Galaxy data from PHANGS survey

PHANGS_data = '../Data/0.PHANGS_v1p6.csv'
PHANGSTable = Table.read(PHANGS_data, format='csv')

PHANGS_clean = PHANGSTable['galaxy', 'orient_ra', 'orient_dec', 'dist', 'orient_posang', 'orient_incl', 'size_r25']

In [6]:
for i in range(len(PHANGSTable)):
    print(PHANGSTable['galaxy', 'survey_alma_status'][i])


 galaxy  survey_alma_status
-------- ------------------
circinus           released
galaxy survey_alma_status
------ ------------------
ic0010 cannot_be_observed
galaxy survey_alma_status
------ ------------------
ic0342 cannot_be_observed
galaxy survey_alma_status
------ ------------------
ic1954           released
galaxy survey_alma_status
------ ------------------
ic1993      not_in_survey
galaxy survey_alma_status
------ ------------------
ic5273           released
galaxy survey_alma_status
------ ------------------
ic5332           released
 galaxy survey_alma_status
------- ------------------
ngc0224 cannot_be_observed
 galaxy survey_alma_status
------- ------------------
ngc0247           released
 galaxy survey_alma_status
------- ------------------
ngc0253           released
 galaxy survey_alma_status
------- ------------------
ngc0278 cannot_be_observed
 galaxy survey_alma_status
------- ------------------
ngc0300           released
 galaxy survey_alma_status
------- --------

In [7]:
SN_plus_gal_data = join(data, PHANGS_clean, keys='galaxy')

# SN_plus_gal_data.colnames

In [8]:
# Pull Telescope Orientation data from PHANGSMapFiles.csv and join to table

telOrientFile = '../Data/0.PHANGSMapFiles.csv'
telOrientTable = Table.read(telOrientFile, format='csv')

ImageFiles150, ErrFiles150, AlphaCOFiles = [],[],[]

for i in range(len(telOrientTable)):
    
    galaxy = telOrientTable['galaxy'][i]
    GalName = galaxy.upper()
    
    if GalName == 'CIRCINUS':
        GalName = 'ESO097-013'
    
    ImageFiles150.append("/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/" + galaxy + "/" + galaxy + "_" + telOrientTable['telOrient'][i] + "_co21_150pc_broad_mom0.fits")
    ErrFiles150.append("/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/" + galaxy + "/" + galaxy + "_" + telOrientTable['telOrient'][i] + "_co21_150pc_broad_emom0.fits")
    AlphaCOFiles.append("/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/" + GalName + '_alphaCO21_Sun+20.fits')

telOrientTable.add_column(ImageFiles150, name = "ImageFile150")
telOrientTable.add_column(ErrFiles150, name = "ErrFile150")
telOrientTable.add_column(AlphaCOFiles, name = "AlphaCOFile")   
    

In [9]:
telOrientTable

galaxy,telOrient,ImageFile150,ErrFile150,AlphaCOFile
str8,str9,str112,str113,str74
circinus,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/ESO097-013_alphaCO21_Sun+20.fits
ic1954,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ic1954/ic1954_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ic1954/ic1954_12m+7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/IC1954_alphaCO21_Sun+20.fits
ic5273,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ic5273/ic5273_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ic5273/ic5273_12m+7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/IC5273_alphaCO21_Sun+20.fits
ic5332,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ic5332/ic5332_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ic5332/ic5332_12m+7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/IC5332_alphaCO21_Sun+20.fits
ngc0247,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0247/ngc0247_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0247/ngc0247_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/NGC0247_alphaCO21_Sun+20.fits
ngc0253,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0253/ngc0253_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0253/ngc0253_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/NGC0253_alphaCO21_Sun+20.fits
ngc0300,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0300/ngc0300_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0300/ngc0300_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/NGC0300_alphaCO21_Sun+20.fits
ngc0628,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0628/ngc0628_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0628/ngc0628_12m+7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/NGC0628_alphaCO21_Sun+20.fits
ngc0685,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0685/ngc0685_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0685/ngc0685_12m+7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/NGC0685_alphaCO21_Sun+20.fits


In [10]:
SN_plus_gal_plus_telOrient_data = join(SN_plus_gal_data,telOrientTable, keys='galaxy')


In [11]:
SN_plus_gal_plus_telOrient_data

galaxy,SN_name,SN_type,SN_ra,SN_dec,SN_xCoord,SN_yCoord,SN_co21int,SN_co21err,SN_co21ew,map_beamsize,orient_ra,orient_dec,dist,orient_posang,orient_incl,size_r25,telOrient,ImageFile150,ErrFile150,AlphaCOFile
str8,str12,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,str112,str113,str74
circinus,SN1996cr,II,213.2918,-65.3457,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078,213.29124,-65.33915,4.2,36.74,64.3,262.49515,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/ESO097-013_alphaCO21_Sun+20.fits
ngc0253,SN1940E,I,11.8783,-25.2934,296.9010534051223,236.26662905273994,219.30367,1.465811,45.862267,0.00232543637022,11.887966,-25.288443,3.7,52.48,75.0,803.7504,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0253/ngc0253_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0253/ngc0253_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/NGC0253_alphaCO21_Sun+20.fits
ngc0300,SN2019qyl,II,13.7399,-37.6444,124.97990204934794,202.45166439134732,-0.41793606,0.4800788,nan,0.002271061982879,13.723024,-37.684475,2.09,114.3,39.8,582.26587,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0300/ngc0300_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0300/ngc0300_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/NGC0300_alphaCO21_Sun+20.fits
ngc0628,SN2019krl,II,24.2068,15.7795,227.48504797816918,732.1448775793053,0.2809551,1.2958647,nan,0.0003114595189268,24.173855,15.783643,9.84,20.7,8.9,296.56592,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0628/ngc0628_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0628/ngc0628_12m+7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/NGC0628_alphaCO21_Sun+20.fits
ngc0628,SN2013ej,II,24.2007,15.7586,333.46022572413716,356.77980642079694,5.7713037,1.9949342,nan,0.0003114595189268,24.173855,15.783643,9.84,20.7,8.9,296.56592,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0628/ngc0628_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0628/ngc0628_12m+7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/NGC0628_alphaCO21_Sun+20.fits
ngc1068,SN2018ivc,II,40.672,-0.0088,41.400000046852824,53.87249997022789,127.77749,0.6721418,32.0753,0.002414131938401,40.66973,-0.0132779,13.97,72.65,34.7,183.28261,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1068/ngc1068_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1068/ngc1068_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/NGC1068_alphaCO21_Sun+20.fits
ngc1087,SN1995V,II,41.6115,-0.4988,161.4198245203906,287.6499560537321,11.513839,1.1399242,5.175495,0.0004451676280474,41.60492,-0.498717,15.85,359.1,42.9,89.14999,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1087/ngc1087_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1087/ngc1087_12m+7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/NGC1087_alphaCO21_Sun+20.fits
ngc1097,SN1999eu,II,41.5866,-30.3184,245.2470996170838,7.223039949207134,0.0,0.7541714,nan,0.0004711325009833,41.578957,-30.274675,13.58,122.4,48.6,317.0452,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1097/ngc1097_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1097/ngc1097_12m+7m+tp_co21_150pc_broad_emom0.fits,/data/

In [12]:
ascii.write(SN_plus_gal_plus_telOrient_data, '../Data/3.SNe+GalData.csv', format='csv', fast_writer=False, overwrite=True)  